In [1]:
import numpy as np
from sentence_transformers import SentenceTransformer
from google import genai
import json
from dotenv import load_dotenv

c:\Users\Koustav Chatterjee\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load environmental variables

load_dotenv()

True

In [3]:
# threshold to address notes not being in RAG
NOT_IN_RAG_THRESHOLD = 0.60

In [4]:
# embedding model

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [5]:
client = genai.Client()

In [6]:
# cosine similarity search


def cosine_similarity(a, b):
    dot_prod = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)

    if norm_a == 0 or norm_b == 0:
        return 0

    return dot_prod / (norm_a * norm_b)

In [7]:
def retrieve(query, embeddings, k=3):
    query_emb = model.encode(query)

    scores = []
    for emb in embeddings:
        score = cosine_similarity(query_emb, np.array(emb["embedding"]))
        scores.append((score, emb))

    scores.sort(reverse=True, key=lambda x: x[0])
    return scores[:k]

In [8]:
# test

with open("json/embeddings.json", "r", encoding="utf-8") as f:
    embeddings = json.load(f)

user_query = "How to calculate postfix expression?"

retrieved = retrieve(user_query, embeddings)
max_score = retrieved[0][0]

if max_score < NOT_IN_RAG_THRESHOLD:
    context = None
else:
    context = "\n\n".join(c["content"] for _, c in retrieved)

prompt = f"""
You are an expert study helper.
Give answer to the user query based on the provided context.
Use the context only if it exists.
If no context is available, say clearly that user has not studied that topic yet.

Think step-by-step carefully and reason internally before answering.
Answer in simple english to the user.

User Query:
{user_query}

{
    f'''Context:
{context}'''
    if context
    else ""
}
"""

response = client.models.generate_content(model="gemini-2.5-flash", contents=prompt)

print(response.text)

To calculate a postfix expression, you can follow these steps:

1.  **Scan from Left to Right**: Look at the expression from the beginning to the end.
2.  **Handle Operands**: If you see a number or a variable (an operand), just keep going.
3.  **Handle Operators**: When you find an operator (like +, -, *, /), apply that operation to the two operands that came just before it.
4.  **Replace and Continue**: Replace those two operands and the operator with the result of your calculation. Now you have one new value, and you continue scanning the rest of the expression with this new value in place.

This process is often done easily using a stack.
